<a href="https://colab.research.google.com/github/subhdeepkaur/comp215/blob/main/labs/lab02-review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COMP 215 - LAB 2
----------------
#### Name(s):
#### Date:

By the end of this lab you should be able to:
  * use strings, tuples, lists, and dictionaries (review)
  * use *f-string* to simplify string formatting operations
  * write functions and simple unit tests (review)
  * use *list comprehension* to implement map and filter algorithms


During this lab, you will be introduced to the following:
  * API Keys
  * JSON data formats
  * the `datetime` module to represent dates and times
  * the `pprint` module to format data structures

## Near Earth Objects

In this lab, we'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/) using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS).  But, as usual, let's do the imports first.


In [37]:
import datetime, json, requests
from datetime import timedelta
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

### dates & f-strings

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [38]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the string
print(formatted_date)

2025-01-16 <class 'datetime.date'>
Today is: 2025-01-16


### Make an API Query

Let's get some data from the NEO database; here's a query that gets the observation "feed" for today.  Notice the first line of the block is an 'API key'.  An API key is a unique identifier that authenticates a user when making a request to an API.  Run the block below using the API key 'DEMO_KEY' (just to be sure it works), then go to https://api.nasa.gov and get your own API key.  Use your own API key for the remainder of the lab.

In [39]:
API_KEY = 'kD4SgSlB5xjFq2OtGum5s5AlwF4nbUxpMJnfm0eo'  # substitute your API key here

# get today's date (as a string)
today = str(datetime.date.today())


# use an f-string to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

# make the API request (recall from lab 1)
response = requests.request("GET", url, headers={}, data={})

# convert the response to json format (recall from lab 1)
data = json.loads(response.text)


# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = len(data)
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos[:1])

3 Near Earth Objects found for 2025-01-16
[{'absolute_magnitude_h': 25.3,
  'close_approach_data': [{'close_approach_date': '2025-01-16',
                           'close_approach_date_full': '2025-Jan-16 20:32',
                           'epoch_date_close_approach': 1737059520000,
                           'miss_distance': {'astronomical': '0.2651162835',
                                             'kilometers': '39660831.313916145',
                                             'lunar': '103.1302342815',
                                             'miles': '24644097.816467001'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '56420.9370068609',
                                                 'kilometers_per_second': '15.6724825019',
                                                 'miles_per_hour': '35057.7761085131'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 169.8341531374,
      

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [40]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')



0 potentially hazardous asteroids identified.


### Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.  We can use this `id` to fetch complete orbital and close approach data for the NEO.  For example, this query fetches the complete data set for the first hazardous asteroid...  Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".


In [41]:
if hazards==[]:
  print("none")
else:
  id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

none
{'absolute_magnitude_h': 25.395,
 'close_approach_data': [{'close_approach_date': '2025-01-09',
                          'close_approach_date_full': '2025-Jan-09 06:09',
                          'epoch_date_close_approach': 1736402940000,
                          'miss_distance': {'astronomical': '0.0211729715',
                                            'kilometers': '3167431.437970705',
                                            'lunar': '8.2362859135',
                                            'miles': '1968150.631197529'},
                          'orbiting_body': 'Earth',
                          'relative_velocity': {'kilometers_per_hour': '70149.0174001931',
                                                'kilometers_per_second': '19.4858381667',
                                                'miles_per_hour': '43587.8713951366'}}],
 'designation': '2025 AU3',
 'estimated_diameter': {'feet': {'estimated_diameter_max': 162.5642444865,
                              

## Exercise 1

In the code cell below, **re-write the accumulator loop** that creates the list of hazards as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [42]:
# Ex. 1 your code here
#list comprehension to get the required list
hazards=[item for item in neos if item['is_potentially_hazardous_asteroid'] is True]
f'{len(hazards)}potentially hazardous asteroids identified.'

'0potentially hazardous asteroids identified.'



## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"


Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [43]:
# Ex. 2 your code here
#function to find the closest object
def closest_approach(close_approach_data):
  closest = None
  min_distance = float('inf')  #taking the max distance as closest

  for approach in close_approach_data:
      if approach['orbiting_body'] == "Earth":
          distance_km = float(approach['miss_distance']['kilometers'])
          if distance_km < min_distance:
              min_distance = distance_km
              closest = (approach['close_approach_date'], distance_km)

  return closest

#function testing
test_data = [
    {'close_approach_date': '2023-01-10', 'orbiting_body': 'Earth', 'miss_distance': {'kilometers': '500000'}},
    {'close_approach_date': '2023-01-11', 'orbiting_body': 'Earth', 'miss_distance': {'kilometers': '300000'}}
]
assert closest_approach(test_data) == ('2023-01-11', 300000.0)
print("Unit test passed.")

Unit test passed.


## Challenge Exercise

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [47]:
# Ex. 3 (challenge) your code here
API_KEY = 'kD4SgSlB5xjFq2OtGum5s5AlwF4nbUxpMJnfm0eo' # Replace with your NASA API key



# Fetching NEOs for the week
today = datetime.date.today()
start_date = (today - timedelta(days=7)).strftime('%Y-%m-%d')
end_date = today.strftime('%Y-%m-%d')

url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={API_KEY}'
response = requests.get(url)
data = json.loads(response.text)

# Extracting NEOs and identifying the closest approach
neos = [neo for day in data['near_earth_objects'].values() for neo in day]
closest_neo = None
closest_date = None
closest_distance = float('inf')

for neo in neos:
    # Fetch complete orbital data
    id = neo['id']
    url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
    response = requests.get(url)
    orbital_data = json.loads(response.text)

    # Determine closest approach
    approach = closest_approach(orbital_data['close_approach_data'])
    if approach and approach[1] < closest_distance:
        closest_neo = neo
        closest_date, closest_distance = approach

# Print details of the closest NEO
if closest_neo:
    print(f"Closest NEO: {closest_neo['name']}")
    print(f"Date of closest approach: {closest_date}")
    print(f"Miss distance: {closest_distance:.2f} km")
else:
    print("No close approaches identified this week.")


Closest NEO: (2021 JE1)
Date of closest approach: 2054-12-10
Miss distance: 109331.49 km
